Exclude the records that SATIVA has highligted, redo the alignment and rebuild the tree.

In [1]:
from reprophylo import *

In [2]:
pj=Project('../1-initial/target_locus.csv', pickle='post_SATIVA.pkl', git=False)

DEBUG:Cloud:Log file (/root/.picloud/cloud.log) opened


In [3]:
pj.read_embl_genbank(['../2-nr/CytB_nr_pre_Sativa.gb'])

/home/reprophylo/reprophylo.py:1015: UserWarning: Version control off
  warnings.warn('Version control off')


In [4]:
pj.extract_by_locus()

Set the stage:

In [5]:
recs_to_keep = {'CytB': []}
recs_to_drop = {'CytB': []}

print "to drop: %s" %len(recs_to_drop['CytB'])
print "to keep: %s" %len(recs_to_keep['CytB'])

print "TOTAL: %s" %(len(recs_to_drop['CytB']) + len(recs_to_keep['CytB']))


to drop: 0
to keep: 0
TOTAL: 0


Record all record ids and assign them the status keep.

In [6]:

for r in pj.records_by_locus['CytB']:
    recs_to_keep['CytB'].append("_".join(r.id.split("_")[:-1]))
    

In [7]:

print "to drop: %s" %len(recs_to_drop['CytB'])
print "to keep: %s" %len(recs_to_keep['CytB'])

print "TOTAL: %s" %(len(recs_to_drop['CytB']) + len(recs_to_keep['CytB']))


to drop: 0
to keep: 1760
TOTAL: 1760


How many do we want to drop? Parse `SATIVA` output and extract records ids putatively mislabeled.

In [8]:
rec_mislabelled = {}
sativa=open('../3-SATIVA/CytB.mis', 'r')

for line in sativa:
    if not line.startswith(";"):
        rec_mislabelled[line.split("\t")[0]] = line.split("\t")[4]
        

In [9]:
print len(rec_mislabelled)

24


Adjust lists accordingly.

In [10]:
for r in rec_mislabelled:
    print r
    if not r in recs_to_drop['CytB']:
        print "\tadd to droplist"
        recs_to_drop['CytB'].append(r)
    for i in reversed(range(len(recs_to_keep['CytB']))):
#        print i
        if r in recs_to_keep['CytB'][i]:
            print "\tremove from keeplist"
            del(recs_to_keep['CytB'][i])
    

EU492281.1
	add to droplist
	remove from keeplist
EU224046.1
	add to droplist
	remove from keeplist
AJ969128.1
	add to droplist
	remove from keeplist
AJ937943.1
	add to droplist
	remove from keeplist
JF489783.1
	add to droplist
	remove from keeplist
KT275288.1
	add to droplist
	remove from keeplist
DQ185405.1
	add to droplist
	remove from keeplist
KT275289.1
	add to droplist
	remove from keeplist
EU224045.1
	add to droplist
	remove from keeplist
JQ346141.1
	add to droplist
	remove from keeplist
JQ661401.1
	add to droplist
	remove from keeplist
AJ937931.1
	add to droplist
	remove from keeplist
AJ937952.1
	add to droplist
	remove from keeplist
AJ937925.1
	add to droplist
	remove from keeplist
JQ661398.1
	add to droplist
	remove from keeplist
AJ937951.1
	add to droplist
	remove from keeplist
KP644340.1
	add to droplist
	remove from keeplist
DQ664351.1
	add to droplist
	remove from keeplist
GQ279764.1
	add to droplist
	remove from keeplist
GU182336.1
	add to droplist
	remove from keeplist


In [11]:

print "to drop: %s" %len(recs_to_drop['CytB'])
print "to keep: %s" %len(recs_to_keep['CytB'])

print "TOTAL: %s" %(len(recs_to_drop['CytB']) + len(recs_to_keep['CytB']))


to drop: 24
to keep: 1736
TOTAL: 1760


Write out good records in full Genbank format including all metadata. Adjust to valid species and taxids before doing it.

Identify records with Subspecies.

In [12]:
from collections import defaultdict

Seqs = []
taxon_to_taxid = {}
recs_to_adjust = defaultdict(list)
taxon_to_recs = defaultdict(list)

for r in pj.records:
    if r.id in recs_to_keep['CytB']:
        Seqs.append(r)
        
        source = [f for f in r.features if f.type == 'source'][0]

        #screen for subspecies and record taxids of valid species
        #if valid species record the taxid
        if (len(source.qualifiers['organism'][0].split(" ")) == 2):
            print source.qualifiers['organism'][0],
            if 'db_xref' in source.qualifiers:
#                print source.qualifiers['db_xref']
                for t in source.qualifiers['db_xref']:
#                    print t
                    if 'taxon' in t:
                        if not source.qualifiers['organism'][0] in taxon_to_taxid:
                            print " .. add to records"
                            taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                        else:
                            print " .. already covered"
                        taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
        #if the species status is 'sp.' but has a valid taxid
        elif (len(source.qualifiers['organism'][0].split(" ")) > 2 and source.qualifiers['organism'][0].split(" ")[1] == 'sp.'):
            print source.qualifiers['organism'][0],
            if 'db_xref' in source.qualifiers:
                for t in source.qualifiers['db_xref']:
                    if 'taxon' in t:
                        if not source.qualifiers['organism'][0] in taxon_to_taxid:
                            print " .. add to records"
                            taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                        else:
                            print " .. already covered" 
                        taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
        #all other cases we consider dubious, e.g. supspecies
        else:
            print "subspecies: %s" %source.qualifiers['organism'][0]
            recs_to_adjust[r.id] = []


Ameiurus melas  .. add to records
Alburnoides bipunctatus  .. add to records
Ambloplites rupestris  .. add to records
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. add to records
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. already covered
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. already covered
Anguilla anguilla  .. already covered
Alburnoides bipunctatus  .. already covered
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. already covered
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. already covered
Anguilla anguilla  .. already covered
Anguilla anguilla  .. already covered
subspecies: Alburnoides bipunctatus strymonicus
Alburnoides bipunctatus  .. already covered
Anguilla anguilla  .. already covered
Anguilla anguilla  .. already covered
subspecies: Alburnoides bipunctatus rossicus
Anguilla anguilla  .. already covered
Anguilla anguilla  .. already covered
Anguilla anguilla  .. alre

Check if any of the taxa that were considered dubious are not already covered in the list after removing subspecies.

In [13]:
from collections import defaultdict

to_fetch = defaultdict(list)

for r in Seqs:
    if r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)
            recs_to_adjust[r.id].append(taxon_to_taxid[adjust_to])
        else:
            to_fetch[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)
            


Alburnoides bipunctatus strymonicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus tzanevi -> Alburnoides bipunctatus
Alburnoides bipunctatus strymonicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus strymonicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus tzanevi -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Abramis brama orientalis -> Abramis brama
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus strymonicus -> Alburnoides bipunctatus
Alburnoides bipunctatus strymonicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoides bipunctatus
Alburnoides bipunctatus rossicus -> Alburnoid

In [14]:
print len(to_fetch)

0


In [15]:
for r in Seqs:
    if r.id in recs_to_adjust:
        print "adjusting %s" %r.id
        source = r.features[0]
        print "%s\t->\t%s" %(source.qualifiers['organism'][0],recs_to_adjust[r.id][0])
        source = r.features[0]
        source.qualifiers['organism'] = [recs_to_adjust[r.id][0]]
        for i in range(len(source.qualifiers['db_xref'])):
            if 'taxon' in source.qualifiers['db_xref'][i]:
                source.qualifiers['db_xref'][i] = "taxon:"+recs_to_adjust[r.id][1]
        if not 'note' in source.qualifiers:
            source.qualifiers['note'] = ['modified from original record: adjusted to species']
        else:
            source.qualifiers['note'].append('modified from original record: adjusted to species')
#        print r.format('genbank')       
#    print r,recs_to_adjust[r]


adjusting KM874614.1
Alburnoides bipunctatus strymonicus	->	Alburnoides bipunctatus
adjusting KM874612.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KM874610.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KM874640.1
Alburnoides bipunctatus tzanevi	->	Alburnoides bipunctatus
adjusting KM874618.1
Alburnoides bipunctatus strymonicus	->	Alburnoides bipunctatus
adjusting KM874606.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KM874617.1
Alburnoides bipunctatus strymonicus	->	Alburnoides bipunctatus
adjusting HM173133.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KM874639.1
Alburnoides bipunctatus tzanevi	->	Alburnoides bipunctatus
adjusting KM874608.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KC894466.1
Abramis brama orientalis	->	Abramis brama
adjusting KM874609.1
Alburnoides bipunctatus rossicus	->	Alburnoides bipunctatus
adjusting KM874615.1
Alburnoides bipunc

Write only good records with all metadata in Genbank format.

In [16]:
from Bio import SeqIO
       
out = open('../CytB_European-fish_SATIVA_cleaned.gb','w')
SeqIO.write(Seqs, out, "genbank")
out.close()

Exclude the records that were highlighted by SATIVA. Then realign and rebuild phylogeny.

In [17]:
pj.exclude(start_from_max=False, **recs_to_drop)

In [20]:
species=[]
for r in pj.records:
    if r.id in recs_to_keep['CytB']:
        source = [f for f in r.features if f.type == 'source'][0]
        species.append(" ".join(source.qualifiers['organism'][0].split()[0:2]))
    
#print set(species)
total=len(species)
species=list(set(species))

print "Total number of records: %i" %total
print "Number of species (see below for full list): %i" %len(species)
print sorted(species)

Total number of records: 1736
Number of species (see below for full list): 65
['Abramis brama', 'Acipenser sturio', 'Alburnoides bipunctatus', 'Alburnus alburnus', 'Alosa fallax', 'Ambloplites rupestris', 'Ameiurus melas', 'Ameiurus nebulosus', 'Anguilla anguilla', 'Aspius aspius', 'Barbatula barbatula', 'Barbus barbus', 'Blicca bjoerkna', 'Carassius auratus', 'Carassius carassius', 'Chondrostoma nasus', 'Cobitis taenia', 'Coregonus albula', 'Coregonus autumnalis', 'Coregonus lavaretus', 'Cottus gobio', 'Ctenopharyngodon idella', 'Cyprinus carpio', 'Esox lucius', 'Gasterosteus aculeatus', 'Gobio gobio', 'Gymnocephalus cernua', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lampetra fluviatilis', 'Lampetra planeri', 'Lepomis gibbosus', 'Leucaspius delineatus', 'Leuciscus idus', 'Leuciscus leuciscus', 'Lota lota', 'Micropterus salmoides', 'Misgurnus fossilis', 'Neogobius melanostomus', 'Oncorhynchus gorbuscha', 'Oncorhynchus mykiss', 'Osmerus eperlanus', 'Perca fluviatilis

Configure Alignment

In [21]:
mafft_linsi = AlnConf(pj,                              # The Project
                    method_name='mafftLinsi-SATIVA', # Any unique method name,
                    CDSAlign=True,                   # Use this method to align
                    codontable=2,                    # The genetic code that
                    program_name='mafft',            # mafft or muscle.
                    cmd='mafft',                     # The command on your machine
                    loci=['CytB'],                 # A list of loci names to align.vel.
                    cline_args={'localpair': True,# Program specific keywords and arguments.
                                'maxiterate': 1000}# cine_args=={} by default, which will
                    )

mafft --localpair --maxiterate 1000 788191473208483.95_CDS_proteins_CytB.fasta


Align.

In [22]:
pj.align([mafft_linsi])

In [23]:
print pj.used_methods['mafftLinsi-SATIVA']

AlnConf named mafftLinsi-SATIVA with ID 788191473208483.95
Loci: CytB 
Created on: Wed Sep  7 00:34:43 2016
Commands:
CytB: mafft --localpair --maxiterate 1000 788191473208483.95_CDS_proteins_CytB.fasta

Environment:
Platform: Linux-3.13.0-54-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.66
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.0
 User: chrishah-office
 Program and version: MAFFT v7.123b\nPal2Nal v14
 Program reference:Katoh
 Standley 2013 (Molecular Biology and Evolution 30:772-780) MAFFT multiple sequence alignment software version 7: improvements in performance and usability.\nMikita Suyama
 David Torrents
 and Peer Bork (2006) PAL2NAL: robust conversion of protein sequence alignments into the corresponding codon alignments.Nucleic Acids Res. 34
 W609-W612.
execution ti

In [24]:
pickle_pj(pj, 'post_SATIVA.pkl')

'post_SATIVA.pkl'

Find custom clipping points

In [25]:
import numpy as np

per_col=[]

aln = pj.fa('CytB@mafftLinsi-SATIVA')

icol=0
for a in aln[1,:]:
    ACGT_count=0
    col=aln[:,icol]
    for i in range(len(col)):
        if col[i] in ['A','G','C','T']:
            ACGT_count+=1
#    print pj.fa('CytB@mafftLinsi')[:,a]
    per_col.append(float(ACGT_count)/len(col))
    icol+=1

wsize=10
min_prop=0.7
    
print "\nFinding forward clip position:\n"
forw_count=0
forw_clip=0
for i in range(len(per_col)-wsize):
        
    if np.mean(per_col[i:i+10]) > min_prop:
        forw_count+=1
        forw_clip=i
        print forw_count,i,np.mean(per_col[i:i+wsize])
    else:
        forw_count=0
        
    if forw_count >= wsize:
        break
        
print "Suggested forward clip at: %s (%s,%s)" %(forw_clip-wsize,forw_count,forw_clip)

print "\nFinding reverse clip position:\n"
reve_count=0
reve_clip=0
for i in reversed(range(len(per_col)-wsize)):
        
    if np.mean(per_col[i:i+10]) > min_prop:
        reve_count+=1
        reve_clip=i
        print reve_count,i,np.mean(per_col[i:i+wsize])
    else:
        reve_count=0
        
    if reve_count >= wsize:
        break
    
print "Suggested reverse clip at: %s (%s,%s)" %(reve_clip+wsize,reve_count,reve_clip)


returning alignment object CytB@mafftLinsi-SATIVA

Finding forward clip position:

1 30 0.706221198157
2 31 0.711693548387
3 32 0.717165898618
4 33 0.723387096774
5 34 0.727476958525
6 35 0.731566820276
7 36 0.735656682028
8 37 0.737960829493
9 38 0.740264976959
10 39 0.743260368664
Suggested forward clip at: 29 (10,39)

Finding reverse clip position:

1 1036 0.702995391705
2 1035 0.706278801843
3 1034 0.709619815668
4 1033 0.712960829493
5 1032 0.716301843318
6 1031 0.719585253456
7 1030 0.719642857143
8 1029 0.719700460829
9 1028 0.720910138249
10 1027 0.722062211982
Suggested reverse clip at: 1037 (10,1027)


In [26]:
from Bio import AlignIO

aln = pj.fa('CytB@mafftLinsi-SATIVA')

clipped_aln = aln[:,forw_clip-wsize:reve_clip+wsize]

pj.trimmed_alignments["CytB@mafftLinsi-SATIVA_aln_clipped"] = clipped_aln

returning alignment object CytB@mafftLinsi-SATIVA


Configure tree building

In [6]:
raxml = RaxmlConf(pj, method_name='raxml-mafftLinsi-SATIVA-clipped', 
                 program_name='raxmlHPC-PTHREADS',
                 keepfiles=True,
                 cmd='raxmlHPC-PTHREADS',
                 preset='fa',
                 alns=['CytB@mafftLinsi-SATIVA_aln_clipped'],
                 model='GAMMA',
                 threads=8,
                 cline_args={'-#': 100})

raxmlHPC-PTHREADS -f a -m GTRGAMMA -n 704791473378903.3_CytB@mafftLinsi-SATIVA_aln_clipped0 -p 295 -s 704791473378903.3_CytB@mafftLinsi-SATIVA_aln_clipped.fasta -T 8 -x 188 -N 100


Infer tree.

In [7]:
pj.tree([raxml])

ApplicationError: Non-zero return code 255 from 'raxmlHPC-PTHREADS -f a -m GTRGAMMA -n 704791473378903.3_CytB@mafftLinsi-SATIVA_aln_clipped0 -p 295 -s 704791473378903.3_CytB@mafftLinsi-SATIVA_aln_clipped.fasta -T 8 -x 188 -N 100'

In [29]:
print pj.used_methods['raxml-mafftLinsi-SATIVA-clipped']

RaxmlConf named raxml-mafftLinsi-SATIVA-clipped with ID 812831473211145.19
Alignments: CytB@mafftLinsi-SATIVA_aln_clipped 
Created on: Wed Sep  7 01:19:05 2016
Commands:
CytB@mafftLinsi-SATIVA_aln_clipped:
raxmlHPC-PTHREADS-SSE3 -f a -m GTRGAMMA -n 812831473211145.19_CytB@mafftLinsi-SATIVA_aln_clipped0 -p 512 -s 812831473211145.19_CytB@mafftLinsi-SATIVA_aln_clipped.fasta -T 5 -x 748 -N 100

Environment:
Platform: Linux-3.13.0-54-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.66
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.0
 User: chrishah-office
 Program and version: raxmlHPC-PTHREADS-SSE3RAxML version 8.0.12
 Program reference: A. Stamatakis: RAxML Version 8: A tool for Phylogenetic Analysis and Post-Analysis of Large Phylogenies. In Bioinformatics
 2014.
execution time:
28332.1

In [30]:
pickle_pj(pj, 'post_SATIVA.pkl')

'post_SATIVA.pkl'

Annotate final tree.

In [31]:
pj.add_qualifier_from_source('organism')


In [32]:
supports = {'black': [100,95],
            'dimgray': [95,75],
            'silver': [75,50]}


In [33]:
for species in ['Lampetra','Petromyzon']:
    pj.if_this_then_that(species, 'organism', 'outgroup', 'outgroup',
                         mode='part')

In [34]:
pj.clear_tree_annotations()
pj.annotate('.', 'outgroup', 'outgroup', 
            ['source_organism', 'record_id'], 
            node_support_dict=supports, multifurc=None,
            html='mafftLinsi-clipped-raxml-post_SATIVA.html',
            fsize=13)

In [35]:
pickle_pj(pj, 'post_SATIVA.pkl')

'post_SATIVA.pkl'

Write out alignment for `pplacer` input. Need to modify record id - remove the '\_f0' that is appended by Reprophylo.

In [13]:
from Bio import AlignIO

aln = pj.fta('CytB@mafftLinsi-SATIVA_aln_clipped')

for r in aln:
    r.id = "_".join(r.id.split("_")[:-1])
    r.description = r.id
    
out=open('../CytB_European-fish_SATIVA_cleaned.alignment.fasta', 'w')
AlignIO.write(aln, out, 'fasta')
out.close()

out=open('../CytB_European-fish_SATIVA_cleaned.alignment.phylip', 'w')
AlignIO.write(aln, out, 'phylip-relaxed')
out.close()


returning trimmed alignment object CytB@mafftLinsi-SATIVA_aln_clipped


Also remove the '\_f0' from the sequence ids in the tree.

In [4]:
%%bash

sed 's/_f[0-9]//g' RAxML_bestTree.812831473211145.19_CytB@mafftLinsi-SATIVA_aln_clipped0 > RAxML_bestTree.CytB@mafftLinsi-SATIVA_aln_clipped0